In [1]:
from cartoframes.auth import set_default_credentials

set_default_credentials('creds.json')

In [2]:
from pandas import read_csv
from geopandas import GeoDataFrame, points_from_xy

remote_file_path = 'http://data.sfgov.org/resource/wg3w-h783.csv'

df = read_csv(remote_file_path)

# Clean latitude and longitude values that are NaN
df = df[df.longitude == df.longitude]
df = df[df.latitude == df.latitude]

incidents_gdf = GeoDataFrame(df, geometry=points_from_xy(df['longitude'], df['latitude']))
incidents_gdf.head()

,incident_datetime,incident_date,incident_time,incident_year,incident_day_of_week,report_datetime,row_id,incident_id,incident_number,cad_number,...,:@computed_region_qgnn_b9vv,:@computed_region_26cr_cadq,:@computed_region_ajp5_b2md,:@computed_region_nqbw_i6c3,:@computed_region_2dwj_jsy4,:@computed_region_h4ep_8xdi,:@computed_region_y6ts_4iup,:@computed_region_jg9y_a9du,:@computed_region_6pnf_4xz7,geometry
0,2019-05-01T01:00:00.000,2019-05-01T00:00:00.000,01:00,2019,Wednesday,2019-06-12T20:27:00.000,81097515200,810975,190424067,191634131.0,...,10.0,7.0,35.0,NaN,NaN,NaN,NaN,NaN,1.0,POINT (-122.49963 37.76257)
1,2019-06-22T07:45:00.000,2019-06-22T00:00:00.000,07:45,2019,Saturday,2019-06-22T08:05:00.000,81465564020,814655,190450880,191730737.0,...,1.0,10.0,34.0,1.0,NaN,1.0,NaN,NaN,2.0,POINT (-122.40816 37.78054)
2,2019-06-03T16:16:00.000,2019-06-03T00:00:00.000,16:16,2019,Monday,2019-06-03T16:16:00.000,80769875000,807698,190397016,191533509.0,...,2.0,9.0,1.0,NaN,NaN,NaN,NaN,NaN,2.0,POINT (-122.39075 37.72160)
3,2018-11-16T16:34:00.000,2018-11-16T00:00:00.000,16:34,2018,Friday,2018-11-16T16:34:00.000,73857915041,738579,180870806,183202539.0,...,6.0,3.0,6.0,NaN,18.0,NaN,NaN,NaN,2.0,POINT (-122.40488 37.79486)
4,2019-05-27T02:25:00.000,2019-05-27T00:00:00.000,02:25,2019,Monday,2019-05-27T02:55:00.000,80509204134,805092,190378555,191470256.0,...,4.0,6.0,13.0,NaN,NaN,NaN,NaN,NaN,1.0,POINT (-122.43056 37.79772)


In [3]:
from cartoframes.viz import Layer

Layer(incidents_gdf)

In [4]:
from geopandas import read_file

neighborhoods_gdf = read_file('https://data.sfgov.org/api/geospatial/pty2-tcw4?method=export&format=GeoJSON')
neighborhoods_gdf.head()

,link,name,geometry
0,"http://en.wikipedia.org/wiki/Sea_Cliff,_San_Fr...",Seacliff,"MULTIPOLYGON (((-122.49346 37.78352, -122.4937..."
1,None,Lake Street,"MULTIPOLYGON (((-122.48715 37.78379, -122.4872..."
2,http://www.nps.gov/prsf/index.htm,Presidio National Park,"MULTIPOLYGON (((-122.47758 37.81099, -122.4771..."
3,None,Presidio Terrace,"MULTIPOLYGON (((-122.47241 37.78735, -122.4710..."
4,http://www.sfgate.com/neighborhoods/sf/innerri...,Inner Richmond,"MULTIPOLYGON (((-122.47263 37.78631, -122.4668..."


In [5]:
Layer(neighborhoods_gdf)

In [6]:
import io
import requests
import os
import zipfile

# Download compressed shapefile
r = requests.get('https://data.sfgov.org/api/geospatial/g8m3-pdis?method=export&format=Shapefile')
z = zipfile.ZipFile(io.BytesIO(r.content))

# Decompress shapefile
os.makedirs('./tmp_dir', exist_ok=True)
z.extractall(path='./tmp_dir')
shp_filename = [filename for filename in os.listdir('./tmp_dir') if filename.endswith('.shp')][0]

# Read shapefile with GeoPandas
business_gdf = read_file('./tmp_dir/{}'.format(shp_filename))
business_gdf.dropna(subset=['geometry'], inplace=True)  # Clean invalid geometries
business_gdf.head()

,business_c,business_z,certificat,city,class_code,date_dba_e,time_dba_e,dba_name,date_dba_s,time_dba_s,...,naic_code_,neighborho,ownership_,parking_ta,pbc_code,state,supervisor,transient_,ttxid,geometry
69,None,94607,0424461,Oakland,07,2018-06-30,00:00:00.000,Falilah Bilal,2008-01-07,00:00:00.000,...,None,None,Bilal Falilah Z,F,7880,CA,None,F,0424461-01-999,POINT (-122.28000 37.81391)
70,None,94549,0383885,Lafayette,07,2015-12-28,00:00:00.000,Arabic Abc,2005-01-10,00:00:00.000,...,None,None,Mokhtar Karim,F,7880,CA,None,F,0383885-04-999,POINT (-122.13382 37.88719)
71,None,94538,0391591,Fremont,13,2015-06-30,00:00:00.000,Consortiumn,2005-09-10,00:00:00.000,...,None,None,Sarnobat S & Jadhav H,F,5001,CA,None,F,0391591-01-999,POINT (-121.97447 37.54600)
72,None,79912,0427559,El Paso,07,2018-06-30,00:00:00.000,Hurry Up Pup,2008-04-20,00:00:00.000,...,None,None,Telles Dave,F,7880,TX,None,F,0427559-01-999,POINT (-106.55340 31.83611)
81,None,92024,0442038,Encinitas,15,2018-06-30,00:00:00.000,Leeger Architecture,2009-08-24,00:00:00.000,...,None,None,Leeger Kathleen O,F,7682,CA,None,F,0442038-01-999,POINT (-117.25635 33.04287)


In [7]:
Layer(business_gdf)

In [8]:
from cartoframes import to_carto

to_carto(incidents_gdf, 'sf_incidents', if_exists='replace')
to_carto(neighborhoods_gdf, 'sf_neighborhoods', if_exists='replace')
to_carto(business_gdf, 'sf_businesses', if_exists='replace')

Success! Data uploaded to table "sf_incidents" correctly
Success! Data uploaded to table "sf_neighborhoods" correctly
Success! Data uploaded to table "sf_businesses" correctly


In [9]:
from cartoframes import read_carto

incidents_neighborhoods_gdf = read_carto("""
    SELECT n.cartodb_id, n.the_geom, n.the_geom_webmercator, n.name, count(*) AS incidents
        FROM sf_incidents i INNER JOIN sf_neighborhoods n
            ON ST_Intersects(i.the_geom, n.the_geom)
        GROUP BY n.cartodb_id
    """)
incidents_neighborhoods_gdf.head()

,cartodb_id,the_geom,name,incidents
0,116,"MULTIPOLYGON (((-122.47157 37.73471, -122.4683...",St. Francis Wood,1
1,87,"MULTIPOLYGON (((-122.39987 37.73029, -122.4008...",Silver Terrace,10
2,71,"MULTIPOLYGON (((-122.46057 37.73043, -122.4605...",Westwood Park,3
3,68,"MULTIPOLYGON (((-122.46376 37.73153, -122.4635...",Mt. Davidson Manor,1
4,51,"MULTIPOLYGON (((-122.44422 37.75558, -122.4442...",Upper Market,3


In [10]:
from cartoframes.auth import get_default_credentials
from cartoframes.data.clients import SQLClient

SQLClient().query("""
    DROP TABLE IF EXISTS {1};
    CREATE TABLE {1} AS
    SELECT n.the_geom, n.name, count(*) AS businesses
        FROM sf_businesses b INNER JOIN sf_neighborhoods n
            ON ST_Intersects(b.the_geom, n.the_geom)
        GROUP BY n.cartodb_id;
    SELECT CDB_CartoDBFyTable('{0}', '{1}')
    """.format(get_default_credentials().username, 'sf_businesses_neighborhoods'))

businesses_neighborhoods_gdf = read_carto('sf_businesses_neighborhoods')
businesses_neighborhoods_gdf.head()

,cartodb_id,the_geom,name,businesses
0,1,"MULTIPOLYGON (((-122.49346 37.78352, -122.4937...",Seacliff,134
1,2,"MULTIPOLYGON (((-122.48715 37.78379, -122.4872...",Lake Street,341
2,3,"MULTIPOLYGON (((-122.47758 37.81099, -122.4771...",Presidio National Park,497
3,4,"MULTIPOLYGON (((-122.47241 37.78735, -122.4710...",Presidio Terrace,464
4,5,"MULTIPOLYGON (((-122.47263 37.78631, -122.4668...",Inner Richmond,3828


In [11]:
from cartoframes.viz import Layout, Map, color_continuous_style

Layout([
    Map([Layer(incidents_neighborhoods_gdf, style=color_continuous_style('incidents'))]),
    Map([Layer(businesses_neighborhoods_gdf, style=color_continuous_style('businesses'))]),
], map_height=500)